In [2]:
import pandas as pd
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}

df = pd.DataFrame(data)
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [3]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
transaction_id={transaction_id}? just give accurate answer in object format \
not provide any extra info & details.

data:
{data}

"""

In [4]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model_name="mistral-small-latest")

response = llm.invoke(prompt)

response.content

'{"payment_status": "Paid"}'

### Specific Functions

In [5]:
import pandas as pd
import json
from pydantic import BaseModel, Field
from langchain.tools import tool

# Define the input schema
class MistralStatusInput(BaseModel):
    df_json: str = Field(description="Serialized DataFrame in JSON format")
    transaction_id: str = Field(description="Transaction ID to look up")

    class Config:
        arbitrary_types_allowed = True

# Tool function to retrieve payment status
@tool(args_schema=MistralStatusInput)
def retrieve_payment_status(df_json: str, transaction_id: str) -> str:
    """Retrieve payment status based on transaction_id."""
    # Deserialize the DataFrame
    df = pd.read_json(df_json)
    if transaction_id in df["transaction_id"].values:
        status = df.loc[df["transaction_id"] == transaction_id, "payment_status"].item()
        return json.dumps({"status": status})
    return json.dumps({"error": "Transaction ID not found."})

# Serialize the DataFrame to JSON
df_json = df.to_json()

# Test the tool
retrieve_payment_status.args


{'df_json': {'description': 'Serialized DataFrame in JSON format',
  'title': 'Df Json',
  'type': 'string'},
 'transaction_id': {'description': 'Transaction ID to look up',
  'title': 'Transaction Id',
  'type': 'string'}}

In [6]:
# Define the input schema for retrieve_payment_date
class PaymentDateInput(BaseModel):
    df_json: str = Field(description="Serialized DataFrame in JSON format")
    transaction_id: str = Field(description="Transaction ID to look up")

    class Config:
        arbitrary_types_allowed = True

# Tool function to retrieve payment date
@tool(args_schema=PaymentDateInput)
def retrieve_payment_date(df_json: str, transaction_id: str) -> str:
    """Retrieve payment date based on transaction_id."""
    # Deserialize the DataFrame
    df = pd.read_json(df_json)
    if transaction_id in df["transaction_id"].values:
        date = df.loc[df["transaction_id"] == transaction_id, "payment_date"].item()
        return json.dumps({"date": date})
    return json.dumps({"error": "Transaction ID not found."})

# Serialize the DataFrame to JSON
df_json = df.to_json()

# Test the tool
retrieve_payment_date.args

retrieve_payment_date({"df_json": df_json, "transaction_id": "T1001"})

C:\Users\Dell\AppData\Local\Temp\ipykernel_17756\2075695128.py:26: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieve_payment_date({"df_json": df_json, "transaction_id": "T1001"})
C:\Users\Dell\AppData\Local\Temp\ipykernel_17756\2075695128.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json)


'{"date": "2021-10-05"}'

In [8]:
def format_tool_to_mistral_function(tool):
    """
    Convert a LangChain tool into a Mistral-compatible function format.
    
    Args:
        tool: The LangChain tool object.

    Returns:
        dict: A Mistral-compatible function representation.
    """
    if not hasattr(tool, "args_schema"):
        raise ValueError("Tool must have an 'args_schema' attribute.")
    
    schema = tool.args_schema.model_json_schema()
    function_name = tool.name
    description = tool.description or "No description available."
    parameters = schema.get("properties", {})

    mistral_function = {
        "name": function_name,
        "description": description,
        "parameters": {
            "type": "object",
            "properties": parameters,
            "required": schema.get("required", []),
        },
    }
    return mistral_function


In [9]:
payment_date_function = format_tool_to_mistral_function(retrieve_payment_date)
print(payment_date_function)


{'name': 'retrieve_payment_date', 'description': 'Retrieve payment date based on transaction_id.', 'parameters': {'type': 'object', 'properties': {'df_json': {'description': 'Serialized DataFrame in JSON format', 'title': 'Df Json', 'type': 'string'}, 'transaction_id': {'description': 'Transaction ID to look up', 'title': 'Transaction Id', 'type': 'string'}}, 'required': ['df_json', 'transaction_id']}}


In [15]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

# Define tools (retrieve_payment_date and retrieve_payment_status are assumed to be defined)
tools = [retrieve_payment_date, retrieve_payment_status]

functions = [format_tool_to_mistral_function(tool) for tool in tools]


# Chat history
chat_history = [
    HumanMessage(role="user", content="What's the status of my transaction T1002?")
]

# Initialize Mistral model
llm = ChatMistralAI(model_name="mistral-large-latest", temperature=0).bind_tools(tools=functions)

# Invoke the model without the `functions` parameter
response = llm.invoke(chat_history)

# Process response
# if "retrieve_payment_status" in response.content:
#     # Call the tool manually
#     result = tools["retrieve_payment_status"](
#         {"df_json": df_json, "transaction_id": "T1002"}
#     )
#     print(result)
# elif "retrieve_payment_date" in response.content:
#     # Call the tool manually
#     result = tools["retrieve_payment_date"](
#         {"df_json": df_json, "transaction_id": "T1002"}
#     )
#     print(result)
# else:
#     print("No matching tool found for the response.")

response


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '9i7Tz2u36', 'type': 'function', 'function': {'name': 'retrieve_payment_status', 'arguments': '{"df_json": "eyJ0cmFuc2FjdGlvbl90aWQiOiAiVDEwMDIifQ==", "transaction_id": "T1002"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 262, 'total_tokens': 329, 'completion_tokens': 67}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-e5930e65-3e85-4ee8-a8c1-960d857c39ec-0', tool_calls=[{'name': 'retrieve_payment_status', 'args': {'df_json': 'eyJ0cmFuc2FjdGlvbl90aWQiOiAiVDEwMDIifQ==', 'transaction_id': 'T1002'}, 'id': '9i7Tz2u36', 'type': 'tool_call'}], usage_metadata={'input_tokens': 262, 'output_tokens': 67, 'total_tokens': 329})

In [18]:
chat_history.append(
    AIMessage(role="assistant", content=response.content)
)
chat_history.append(HumanMessage(role="user", content="My transaction ID is T1001."))
chat_history

[HumanMessage(content="What's the status of my transaction?", additional_kwargs={}, response_metadata={}, role='user'),
 AIMessage(content='Please provide me with the transaction ID.', additional_kwargs={}, response_metadata={}, role='assistant'),
 HumanMessage(content='My transaction ID is T1001.', additional_kwargs={}, response_metadata={}, role='user'),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, role='assistant'),
 HumanMessage(content='My transaction ID is T1001.', additional_kwargs={}, response_metadata={}, role='user')]

In [17]:
response = llm.invoke(chat_history)
response.content

''